<a href="https://colab.research.google.com/github/benvictoria21/python-machine-learning/blob/master/RNN_Simplified_(Google_Stock_Price_Forecasting).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import warnings  
warnings.filterwarnings('ignore') # to ignore the warnings

In [ ]:
training = pd.read_csv("https://raw.githubusercontent.com/raj-shah14/Predicting-Google-Stock-Prices/master/Google_Stock_Price_Train.csv")
training.head()

In [ ]:
real_stock_price_train = training.iloc[:, 1:2].values     # creates a 2D array having observation and feature

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training2 = sc.fit_transform(real_stock_price_train)

In [ ]:
training2.shape

In [ ]:
# hence in the input we take
X_train = training2[0:1257]  # all but last observation as we don't have the output value for it
y_train = training2[1:1258]  # values shifted by 1

In [ ]:
X_train = np.reshape(X_train, (1257, 1, 1))
# (1257, 1, 1) the 2nd argument is no. of features and 3rd argument is the time step

In [ ]:
# importing libraries
from keras.models import Sequential  # initialize NN as a sequnce of layers
from keras.layers import Dense  # to add fully connected layers
from keras.layers import LSTM

In [ ]:
rnn_regressor = Sequential()

In [ ]:
rnn_regressor.add(LSTM(units=4, activation='sigmoid', input_shape=(1, 1)))

In [ ]:
rnn_regressor.add(Dense(units=1))

In [ ]:
rnn_regressor.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
rnn_regressor.fit(X_train, y_train, batch_size=32, epochs=200)

In [ ]:
# predicting the training results
predicted_stock_price_train = rnn_regressor.predict(X_train)
predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)

In [ ]:
# visualizing the training results
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
plt.plot(real_stock_price_train, color = 'red', label='Real Google Stock Price')
plt.plot(predicted_stock_price_train, color = 'blue', label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
testing = pd.read_csv("https://raw.githubusercontent.com/Parasgr7/Google-Stock-Price-Prediction/master/Google_Stock_Price_Test.csv")
testing.head()

In [ ]:
# taking the column of "open" value of stock price
real_stock_price_test = testing.iloc[:, 1:2].values


In [ ]:
# feature Scaling
inputs = sc.transform(real_stock_price_test)

In [ ]:
# reshaping
inputs = np.reshape(inputs, (20, 1, 1))     # only 20 observations in testing set


In [ ]:
# predicting the stock price (for the year 2017)
predicted_stock_price_test = rnn_regressor.predict(inputs)     # but these are the scaled values

In [ ]:
predicted_stock_price_test = sc.inverse_transform(predicted_stock_price_test)

In [ ]:
# visualizing the results for testing
plt.figure(figsize=(20,10))
plt.plot(real_stock_price_test, color = 'red', label='Real Google Stock Price')
plt.plot(predicted_stock_price_test, color = 'blue', label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction (Test Set)')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(real_stock_price_test, predicted_stock_price_test))
print('The RMSE value is', rmse)

In [ ]:
print('RMSE in terms of % of the orignal value is', round((rmse/real_stock_price_test.mean()*100), 2) , '%')   
# we take the avg because it would be a true representative of the real stock values